Importing required libraries

In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm
import copy
import time

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torchvision import models
from torch import optim, nn
import torch.nn.functional as F

In [4]:
device = 'cpu'

if torch.cuda.is_available() :
    device = 'cuda'

In [5]:
device

'cuda'

Creating dataset

In [6]:
class ImageDataset(Dataset) :

    def __init__(self, transform) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        # Transformations
        self.transforms = transform

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in enumerate(self.domains) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)

                    # One hot encoding domain
                    no_domains = 6
                    domainVector = np.zeros(no_domains)
                    domainVector[-1] = 1
                    domainVector[i_dom] = 1
                    self.domains_y.append(domainVector)

                    # One hot encoding class
                    classVector = np.zeros(7)
                    classVector[i_cla] = 1
                    self.classes_y.append(classVector)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)

    def __getitem__(self, index) :

        return self.transforms(self.images[index].astype('float')/255), self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return len(self.images)

Defining transforms

In [7]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(50),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [8]:
dataset = ImageDataset(transform=transform)

In [9]:
# Train and test split
train_dataset, val_dataset, test_dataset = random_split(dataset, [6000, 1000, 2991])

In [10]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

In [11]:
dataloaders = {
    'train' : train_dataloader,
    'val' : val_dataloader,
    'test' : test_dataloader
}

dataset_sizes = {
    'train' : 6000,
    'val': 1000,
    'test' : 2991
}

In [12]:
def train_model(model, criterion, optimizer, epochs=1):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 0.0
    best_acc = 0
    
    for ep in range(epochs):
        print(f"Epoch {ep}/{epochs-1}")
        print("-"*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
                
            for images, domains, labels in tqdm(dataloaders[phase]):
                images = images.to(device)
                labels = labels.to(device)
                domains = domains.to(device)
                _, labels_list = torch.max(labels, 1)
                labels_list = labels_list.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images.float(), domains.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels_list)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f"{phase} Loss:{epoch_loss:.4f} Acc:{epoch_acc:.4f}")
            
            if phase == 'val':
                if ep == 0:
                    best_loss = epoch_loss
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    if epoch_loss < best_loss:
                        best_loss = epoch_loss
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time() - since
    
    print(f'Training complete in {time_elapsed // 60}m {time_elapsed % 60}s')
    print(f'Best val loss: {best_loss:.4f}')
    print(f'Best acc: {best_acc}')
    
    model.load_state_dict(best_model_wts)
    
    return model

In [13]:
class customLinear(nn.Module) :
    def __init__(self, in_feat, out_feat) :
        super().__init__()

        linMod = nn.Linear(in_feat, out_feat)

        self.weight = nn.Parameter(linMod.weight.detach())
        self.bias = nn.Parameter(linMod.bias)

    def forward(self, X) :
        output = (X@self.weight.T)+self.bias
        return output

In [14]:
def getWeightsAndBiases(in_size, out_size, no_domains) :

    weight_mat = torch.zeros(out_size, in_size, no_domains+1)
    bias_mat = torch.zeros(out_size, no_domains+1)

    for i in range(no_domains+1) :
        linMod = nn.Linear(in_size, out_size)

        weight = linMod.weight.detach()
        bias = linMod.bias.detach()

        if not(i == no_domains) :
          weight_mat[:, :, i] = weight/no_domains
          bias_mat[:, i] = bias/no_domains

    return weight_mat, bias_mat

In [15]:
class Model(nn.Module):
    def __init__(self):
        
        super().__init__()

        no_domains = 5
        
        # Inititializing weights and biases
        weights1, biases1 = getWeightsAndBiases(3*50*50, 2048, no_domains)
        weights2, biases2 = getWeightsAndBiases(2048, 2048, no_domains)
        weights3, biases3 = getWeightsAndBiases(2048, 1024, no_domains)
        weights4, biases4 = getWeightsAndBiases(1024, 256, no_domains)
        weights5, biases5 = getWeightsAndBiases(256, 7, no_domains)

        # make weights torch parameters
        self.weights1 = nn.Parameter(weights1)
        self.weights2 = nn.Parameter(weights2)
        self.weights3 = nn.Parameter(weights3)
        self.weights4 = nn.Parameter(weights4)
        self.weights5 = nn.Parameter(weights5)    

        # make biases torch parameters
        self.biases1 = nn.Parameter(biases1)
        self.biases2 = nn.Parameter(biases2)
        self.biases3 = nn.Parameter(biases3)
        self.biases4 = nn.Parameter(biases4)
        self.biases5 = nn.Parameter(biases5)    
        
    def forward(self, X, X_dom):
        # Flattening the given image
        X_flat = X.reshape(-1, 3*50*50)

        weights1 = torch.inner(self.weights1, X_dom.reshape(-1))
        weights2 = torch.inner(self.weights2, X_dom.reshape(-1))
        weights3 = torch.inner(self.weights3, X_dom.reshape(-1))
        weights4 = torch.inner(self.weights4, X_dom.reshape(-1))
        weights5 = torch.inner(self.weights5, X_dom.reshape(-1))

        biases1 = torch.inner(self.biases1, X_dom.reshape(-1))
        biases2 = torch.inner(self.biases2, X_dom.reshape(-1))
        biases3 = torch.inner(self.biases3, X_dom.reshape(-1))
        biases4 = torch.inner(self.biases4, X_dom.reshape(-1))
        biases5 = torch.inner(self.biases5, X_dom.reshape(-1))

        out1 = ((X_flat@weights1.T)+biases1)
        out1 = F.relu(out1)
        out2 = ((out1@weights2.T)+biases2)
        out2 = F.relu(out2)
        out3 = ((out2@weights3.T)+biases3)
        out3 = F.relu(out3)
        out4 = ((out3@weights4.T)+biases4)
        out4 = F.relu(out4)
        out5 = ((out4@weights5.T)+biases5)
        out6 = F.softmax(out5, dim=1)

        return out6

# Initializing model
model = Model()

# To GPU
model = model.to(device)

# Defining loss function
criterion = nn.CrossEntropyLoss()

# Defining optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [16]:
model = train_model(model, criterion, optimizer, 10)

Epoch 0/9
----------


100%|██████████| 6000/6000 [11:56<00:00,  8.38it/s]


train Loss:1.9399 Acc:0.1760


100%|██████████| 1000/1000 [00:54<00:00, 18.46it/s]


val Loss:1.9354 Acc:0.1930

Epoch 1/9
----------


...

val Loss:1.8600 Acc:0.2600

Epoch 9/9
----------


100%|██████████| 6000/6000 [11:52<00:00,  8.42it/s]


train Loss:1.8465 Acc:0.2823


100%|██████████| 1000/1000 [00:52<00:00, 18.93it/s]

val Loss:1.8592 Acc:0.2700

Training complete in 128.0m 10.487203121185303s
Best val loss: 1.8592
Best acc: 0.27
